In [2]:
# coding:utf8
import numpy as np
import math
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot
from bokeh.models import LinearAxis, LogAxis, Range1d
from scipy.fftpack import fft, fftfreq, fftshift, rfft
output_file('Fourier.html')
%matplotlib notebook

In [6]:
tick = pd.read_csv('RIM6_4.trd', usecols=(0, 1, 2, 3, 4, 5, 6),
                   parse_dates={'DateTime': [0,1]}, index_col=False)
tick = tick.rename(columns={'<LAST>': 'Price', '<VOL>': 'Vol', '<DIRECTION>': 'Dir'})
tick.ix[tick.Dir == 'Sell', 'VolSell'] = tick.Vol
tick.ix[tick.Dir == 'Buy', 'VolBuy'] = tick.Vol

In [7]:
Nticks = 500     # Количество тиков для бара и ресэмплинг по тикам
Nback = 64
Ndeltas = 500
TimeRes = '1s'  # Интервал ресэмплинга в секундах
dt = 1.0        # Interval FFT
fs = 1.0/dt      # Sampling frequency
dl = tick.fillna(0)
dl['CumDelta'] = (dl['VolBuy'] - dl['VolSell']).cumsum()
dl['AbsCumDelta'] = (abs(dl['CumDelta']))/Ndeltas
dl

,DateTime,<MSEC>,<TRADENO>,Price,Vol,Dir,VolSell,VolBuy,CumDelta,AbsCumDelta
0,2016-04-20 10:00:00,1,1462284886,91860,1,Sell,1.0,0.0,-1.0,0.002
1,2016-04-20 10:00:00,1,1462284887,91840,2,Sell,2.0,0.0,-3.0,0.006
2,2016-04-20 10:00:00,1,1462284888,91830,1,Sell,1.0,0.0,-4.0,0.008
3,2016-04-20 10:00:00,1,1462284889,91750,1,Sell,1.0,0.0,-5.0,0.010
4,2016-04-20 10:00:00,1,1462284890,91700,3,Sell,3.0,0.0,-8.0,0.016
5,2016-04-20 10:00:00,1,1462284891,91700,1,Sell,1.0,0.0,-9.0,0.018
6,2016-04-20 10:00:00,1,1462284892,91680,1,Sell,1.0,0.0,-10.0,0.020
7,2016-04-20 10:00:00,1,1462284893,91670,1,Sell,1.0,0.0,-11.0,0.022
8,2016-04-20 10:00:00,1,1462284894,91650,4,Sell,4.0,0.0,-15.0,0.030
9,2016-04-20 10:00:00,1,1462284895,91610,1,Sell,1.0,0.0,-16.0,0.032


In [8]:
rows_list = []
mx = int(math.ceil(dl['AbsCumDelta'].max()))
dl.set_index('AbsCumDelta')
for i in range(0, mx):
    tmp = dl.loc[(dl['AbsCumDelta'] > float(i)) & (dl['AbsCumDelta'] <= float(i + 1))]
    tmp.set_index('Price')
    dict1 = {}

    op = tmp.Price.iloc[0]
    hi = tmp.Price.max()
    lo = tmp.Price.min()
    cl = tmp.Price.iloc[(len(tmp.index) - 1)]
    cumdelta = tmp.CumDelta.iloc[(len(tmp.index) - 1)]
    vBuy = tmp.VolBuy.sum()
    vSell = tmp.VolSell.sum()
    nBuy = tmp.VolBuy.count()
    nSell = tmp.VolSell.count()
    tClose = tmp.DateTime.max()
    dtime = (tmp.DateTime.max() - tmp.DateTime.min())
    dict1.update({'DateTime': tClose, 'open': op, 'high': hi,
                  'low': lo, 'close': cl, 'TickBuy': nBuy,
                  'VolBuy': vBuy, 'TickSell': nSell, 'VolSell': vSell, 'DeltaTime': dtime, 'CumDelta': cumdelta})
    rows_list.append(dict1)

res = pd.DataFrame(rows_list)

res['Dsec'] = res['DeltaTime'].dt.total_seconds()
res['TSpeedBuy'] = res['TickBuy'] / res['Dsec']
res['TSpeedSell'] = res['TickSell'] / res['Dsec']
res['VSpeedBuy'] = res['VolBuy'] / res['Dsec']
res['VSpeedSell'] = res['VolSell'] / res['Dsec']
res['OTO'] = ((res['TickBuy'] - res['TickSell']) / (res['TickBuy'] + res['TickSell']) * 0.65
              + (res['VolBuy'] - res['VolSell']) / (res['VolBuy'] + res['VolSell']) * 0.35) * 100.0
res['OTON'] = ((res['TickBuy'] * res['VolBuy'] - res['TickSell'] * res['VolSell']) /
               (res['TickBuy'] * res['VolBuy'] + res['TickSell'] * res['VolSell'])) * 100.0
res

,CumDelta,DateTime,DeltaTime,TickBuy,TickSell,VolBuy,VolSell,close,high,low,open,Dsec,TSpeedBuy,TSpeedSell,VSpeedBuy,VSpeedSell,OTO,OTON
0,499.0,2016-04-20 12:33:31,02:33:31,20609,20609,28846.0,28264.0,92280,92340,91010,91860,9211.0,2.237434,2.237434,3.131690,3.068505,0.356680,1.019086
1,992.0,2016-04-20 12:33:47,02:33:47,21725,21725,30286.0,29894.0,92340,92380,90910,91010,9227.0,2.354503,2.354503,3.282324,3.239840,0.227983,0.651379
2,1453.0,2016-04-20 12:38:16,02:38:15,19230,19230,27338.0,26911.0,92330,92430,90700,90900,9495.0,2.025276,2.025276,2.879200,2.834229,0.275489,0.787111
3,1999.0,2016-04-20 12:40:42,02:40:41,16865,16865,23960.0,23782.0,92310,92470,90620,90820,9641.0,1.749300,1.749300,2.485219,2.466757,0.130493,0.372837
4,2440.0,2016-04-20 12:42:12,02:42:09,14752,14752,19672.0,19248.0,92310,92500,90550,90620,9729.0,1.516291,1.516291,2.021996,1.978415,0.381295,1.089414
5,3000.0,2016-04-20 12:42:38,02:42:21,13583,13583,19254.0,18214.0,92380,92500,90670,90700,9741.0,1.394415,1.394415,1.976594,1.869829,0.971496,2.775702
6,3500.0,2016-04-20 12:47:08,02:46:12,11994,11994,16469.0,15860.0,92450,92550,90810,90900,9972.0,1.202768,1.202768,1.651524,1.590453,0.659315,1.883758
7,3951.0,2016-04-20 12:47:14,02:46:09,9771,9771,13139.0,12962.0,92500,92550,90750,90820,9969.0,0.980138,0.980138,1.317986,1.300231,0.237347,0.678135
8,4499.0,2016-04-20 12:48:14,02:46:14,7165,7165,10427.0,9697.0,92500,92530,90720,90810,9974.0,0.718368,0.718368,1.045418,0.972228,1.269628,3.627509
9,4999.0,2016-04-20 12:54:10,02:51:56,6978,6978,9633.0,9034.0,92560,92580,90660,90740,10316.0,0.676425,0.676425,0.933792,0.875727,1.123105,3.208871


In [39]:
# res = res.set_index('DateTime')
mids = (res.open + res.close)/2
spans = abs(res.close - res.open)
inc = res.close > res.open
dec = res.open > res.close
#w = 12*60*60
w = 0.5

In [42]:
p = figure(x_range=(0, len(res.index)), plot_width=800, plot_height=600, title='RIM6 CandleStick')
p.y_range=Range1d(start=90500, end=95000)
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha = 0.3
p.segment(res.index.values, res.high, res.index.values, res.low, color='black')
p.rect(res.index[inc], mids[inc], w, spans[inc], fill_color='#6CF350', line_color='green')
p.rect(res.index[dec], mids[dec], w, spans[dec], fill_color='#FF0000', line_color='red')
p.extra_y_ranges = {}
p.extra_y_ranges['TickSpeed'] = Range1d(start=0, end=100)
p.extra_y_ranges['OTO'] = Range1d(start=-60, end=60)
p.extra_y_ranges['OTON'] = Range1d(start=-60, end=60)
p.extra_y_ranges['CumDelta'] = Range1d(start=-3000, end=52000)
#p.line(res.index, res.TSpeedBuy, color='blue', y_range_name='TickSpeed', alpha=0.5)
p.line(res.index, res.TSpeedBuy.rolling(window=4).median(), color='green', y_range_name='TickSpeed', alpha=0.8)
#p.line(res.index, res.TSpeedSell, color='red', y_range_name='TickSpeed', alpha=0.5)
p.line(res.index, res.TSpeedSell.rolling(window=9).median(), color='red', y_range_name='TickSpeed', alpha=0.8)
p.line(res.index, res.OTO.rolling(window=15).mean(), color='brown', y_range_name='OTO', alpha=0.9)
p.line(res.index, res.OTON.rolling(window=15).mean(), color='blue', y_range_name='OTON', alpha=0.9)
p.line(res.index, res.CumDelta, color='black', y_range_name='CumDelta', alpha=0.9)
p.add_layout(LogAxis(y_range_name='TickSpeed'), 'left')
p.add_layout(LinearAxis(y_range_name='OTO'), 'left')
p.add_layout(LinearAxis(y_range_name='OTON'), 'left')
p.add_layout(LinearAxis(y_range_name='CumDelta'), 'left')
#show(p)
#l=figure(plot_width=800, plot_height=600, title='FFT')
#l.line(resfftfreq, resfft)
#m = gridplot([[p, l]])

show(p)